In [293]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from shapely.ops import nearest_points
import numpy as np

#pd.set_option('display.max_columns', None)

In [294]:
master = gpd.read_file('export/master.geojson')
accidents = gpd.read_file('export/accidents/accidents.geojson')

In [295]:
master['GDE_CNTR'] = master.apply(lambda row: Point(row['E_CNTR'], row['N_CNTR']), axis=1)

/Users/Dario/opt/anaconda3/envs/GTA22/lib/python3.9/site-packages/pandas/core/dtypes/cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [296]:
master['smaller_2'] = 0
master['2_to_5'] = 0
master['5_to_10'] = 0

In [297]:
mas = master
acc = accidents

In [298]:
import numpy as np
import geopandas as gpd

def calc_distance_vectorized(gdf1, gdf2):
    return gdf1.geometry.apply(lambda x: gdf2.distance(x))

def vectorized(mas, acc):
    # calculate distances between mas and acc geometries using the vectorized function
    distances = calc_distance_vectorized(mas, acc)

    # count the number of distances that fall into each distance category using boolean indexing
    mas['smaller_2'] = (distances < 2000).sum(axis=1)
    mas['2_to_5'] = ((distances >= 2000) & (distances < 5000)).sum(axis=1)
    mas['5_to_10'] = ((distances >= 5000) & (distances < 10000)).sum(axis=1)

    return mas


In [291]:
def calc_distance(p1, p2):
    return p1.distance(p2)

def for_loop(mas, acc):
    for i, row1 in mas.iterrows():
        for j, row2 in acc.iterrows():
            dist = calc_distance(row1.GDE_CNTR, row2.geometry)
            if dist < 2000:
                mas.at[i, 'smaller_2'] += 1
            elif dist < 5000:
                mas.at[i, '2_to_5'] += 1
            elif dist < 10000:
                mas.at[i, '5_to_10'] += 1
            else:
                pass
    return mas

In [299]:
vector = vectorized(mas, acc)
#for_l = for_loop(mas, acc)

In [300]:
vector = vector.drop(columns='GDE_CNTR', axis=1)
#for_l = for_l.drop(columns='GDE_CNTR', axis=1)

In [302]:
vector.to_file('export/master_with_acc.geojson', driver='GeoJSON')
#for_l.to_file('for.geojson', driver='GeoJSON')